# Spare Part Demand Forecasting - Exploratory Data Analysis

This notebook explores the spare parts demand dataset to understand patterns, trends, and prepare for model development.

## Objectives
1. Load and understand the dataset structure
2. Analyze demand patterns (daily, weekly, monthly, yearly)
3. Explore category and service center distributions
4. Identify seasonality and trends
5. Prepare insights for feature engineering

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print('Libraries loaded successfully!')

## 1. Load Data

In [ ]:
# Load the dataset
df = pd.read_csv('../data/raw/spare_parts_demand.csv', parse_dates=['date'])

print(f'Dataset Shape: {df.shape}')
print(f'Date Range: {df["date"].min()} to {df["date"].max()}')
print(f'\nColumns: {list(df.columns)}')

In [ ]:
# Display first few rows
df.head(10)

In [ ]:
# Data types and info
df.info()

In [ ]:
# Summary statistics
df.describe()

## 2. Data Quality Check

In [ ]:
# Check for missing values
missing = df.isnull().sum()
print('Missing Values:')
print(missing[missing > 0] if any(missing > 0) else 'No missing values!')

In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f'Duplicate rows: {duplicates}')

In [ ]:
# Unique values
print('Unique Values:')
print(f'  Parts: {df["part_id"].nunique()}')
print(f'  Categories: {df["category"].nunique()}')
print(f'  Service Centers: {df["service_center"].nunique()}')
print(f'  Days: {df["date"].nunique()}')

## 3. Demand Distribution Analysis

In [ ]:
# Demand distribution
fig = make_subplots(rows=1, cols=2, subplot_titles=('Demand Distribution', 'Log Demand Distribution'))

fig.add_trace(
    go.Histogram(x=df['demand_quantity'], nbinsx=50, name='Demand', marker_color='#F97316'),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=np.log1p(df['demand_quantity']), nbinsx=50, name='Log Demand', marker_color='#3B82F6'),
    row=1, col=2
)

fig.update_layout(title_text='Demand Quantity Distribution', showlegend=False, height=400)
fig.show()

In [ ]:
# Box plot by category
fig = px.box(df, x='category', y='demand_quantity', color='category',
             title='Demand Distribution by Category',
             color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_layout(showlegend=False)
fig.show()

## 4. Time Series Analysis

In [ ]:
# Aggregate daily demand
daily_demand = df.groupby('date')['demand_quantity'].sum().reset_index()

fig = px.line(daily_demand, x='date', y='demand_quantity',
              title='Total Daily Demand Over Time',
              color_discrete_sequence=['#F97316'])
fig.update_layout(xaxis_title='Date', yaxis_title='Total Demand')
fig.show()

In [ ]:
# Monthly trend
df['year_month'] = df['date'].dt.to_period('M').astype(str)
monthly_demand = df.groupby('year_month')['demand_quantity'].sum().reset_index()

fig = px.bar(monthly_demand, x='year_month', y='demand_quantity',
             title='Monthly Demand Trend',
             color='demand_quantity', color_continuous_scale='Oranges')
fig.update_layout(xaxis_title='Month', yaxis_title='Total Demand')
fig.show()

In [ ]:
# Weekly pattern (day of week)
dow_demand = df.groupby('day_of_week')['demand_quantity'].mean().reset_index()
dow_demand['day_name'] = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

fig = px.bar(dow_demand, x='day_name', y='demand_quantity',
             title='Average Demand by Day of Week',
             color='demand_quantity', color_continuous_scale='Oranges')
fig.update_layout(xaxis_title='Day of Week', yaxis_title='Average Demand')
fig.show()

In [ ]:
# Monthly seasonality
month_demand = df.groupby('month')['demand_quantity'].mean().reset_index()
month_demand['month_name'] = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig = px.line(month_demand, x='month_name', y='demand_quantity',
              title='Average Demand by Month (Seasonality)',
              markers=True, color_discrete_sequence=['#F97316'])
fig.update_layout(xaxis_title='Month', yaxis_title='Average Demand')
fig.show()

## 5. Category Analysis

In [ ]:
# Total demand by category
category_demand = df.groupby('category')['demand_quantity'].sum().reset_index()
category_demand = category_demand.sort_values('demand_quantity', ascending=False)

fig = px.pie(category_demand, values='demand_quantity', names='category',
             title='Demand Distribution by Category',
             color_discrete_sequence=px.colors.qualitative.Set2)
fig.show()

In [ ]:
# Category trend over time
category_monthly = df.groupby(['year_month', 'category'])['demand_quantity'].sum().reset_index()

fig = px.line(category_monthly, x='year_month', y='demand_quantity', color='category',
              title='Monthly Demand Trend by Category')
fig.update_layout(xaxis_title='Month', yaxis_title='Total Demand')
fig.show()

## 6. Service Center Analysis

In [ ]:
# Demand by service center
sc_demand = df.groupby('service_center')['demand_quantity'].sum().reset_index()

fig = px.bar(sc_demand, x='service_center', y='demand_quantity',
             title='Total Demand by Service Center',
             color='service_center', color_discrete_sequence=px.colors.qualitative.Bold)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Heatmap: Service Center vs Category
sc_cat = df.groupby(['service_center', 'category'])['demand_quantity'].sum().unstack(fill_value=0)

fig = px.imshow(sc_cat, text_auto=True, color_continuous_scale='Oranges',
                title='Demand Heatmap: Service Center vs Category',
                labels={'x': 'Category', 'y': 'Service Center', 'color': 'Demand'})
fig.show()

## 7. Top Parts Analysis

In [ ]:
# Top 10 parts by demand
top_parts = df.groupby(['part_id', 'part_name', 'category'])['demand_quantity'].sum().reset_index()
top_parts = top_parts.nlargest(10, 'demand_quantity')

fig = px.bar(top_parts, x='demand_quantity', y='part_name', orientation='h',
             title='Top 10 Parts by Total Demand',
             color='category', color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

In [ ]:
# Top 10 parts by revenue
top_revenue = df.groupby(['part_id', 'part_name', 'category'])['revenue'].sum().reset_index()
top_revenue = top_revenue.nlargest(10, 'revenue')

fig = px.bar(top_revenue, x='revenue', y='part_name', orientation='h',
             title='Top 10 Parts by Total Revenue',
             color='category', color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

## 8. Correlation Analysis

In [ ]:
# Correlation matrix
numeric_cols = ['demand_quantity', 'unit_price', 'revenue', 'day_of_week', 'month', 'quarter']
corr_matrix = df[numeric_cols].corr()

fig = px.imshow(corr_matrix, text_auto='.2f', color_continuous_scale='RdBu_r',
                title='Correlation Matrix')
fig.show()

## 9. Key Insights Summary

In [ ]:
print('='*60)
print('KEY INSIGHTS FROM EDA')
print('='*60)

print(f'''\n1. DATASET OVERVIEW\n   - Total records: {len(df):,}\n   - Date range: 2 years (2022-2023)\n   - Parts tracked: {df['part_id'].nunique()}\n   - Service centers: {df['service_center'].nunique()}\n   - Categories: {df['category'].nunique()}')

print(f'''\n2. DEMAND PATTERNS\n   - Average daily demand: {df.groupby('date')['demand_quantity'].sum().mean():.0f} units\n   - Peak demand month: {month_demand.loc[month_demand['demand_quantity'].idxmax(), 'month_name']}\n   - Lowest demand day: Weekend (Sat/Sun)')

print(f'''\n3. TOP PERFORMERS\n   - Highest demand category: {category_demand.iloc[0]['category']}\n   - Highest demand SC: {sc_demand.loc[sc_demand['demand_quantity'].idxmax(), 'service_center']}')

print('''\n4. RECOMMENDATIONS FOR MODELING\n   - Use lag features (7, 14, 30 days)\n   - Include day of week as feature\n   - Add monthly seasonality\n   - Consider category-specific models\n   - Prophet for long-term, XGBoost for short-term''')

print('\n' + '='*60)

## 10. Save Processed Data for Modeling

In [ ]:
# Aggregate daily demand for time series modeling
daily_agg = df.groupby('date').agg({
    'demand_quantity': 'sum',
    'revenue': 'sum'
}).reset_index()

daily_agg.to_csv('../data/processed/daily_demand.csv', index=False)
print(f'Saved daily aggregated data: {daily_agg.shape}')

# Save part-level daily data
part_daily = df.groupby(['date', 'part_id', 'category']).agg({
    'demand_quantity': 'sum',
    'revenue': 'sum'
}).reset_index()

part_daily.to_csv('../data/processed/part_daily_demand.csv', index=False)
print(f'Saved part-level daily data: {part_daily.shape}')

In [ ]:
print('EDA Complete! Ready for model development.')